In [3]:
import warnings
warnings.filterwarnings("ignore")
%pylab inline
import pandas as pd
#import graphviz

Populating the interactive namespace from numpy and matplotlib


In [9]:
data = pd.read_csv("winequality-red.csv", header=0)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [18]:
x_data =  data.iloc[:,0:(len(data.columns)-2)]
y_data =  data.iloc[:,len(data.columns)-1]
#y_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56


In [56]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20, random_state=33)

In [60]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=20 ,min_samples_leaf=10)
clf = clf.fit(x_train,y_train)
dp = clf.decision_path(x_train)
#print(dp)

In [61]:
#gdata = tree.export_graphviz(clf, out_file="dt.png", filled=True, rounded=True)
#graph = graphviz.Source(gdata)

In [62]:
from sklearn import metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
    y_pred=clf.predict(X)   
    if show_accuracy:
        print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n")

    if show_classification_report:
        print ("Classification report")
        print (metrics.classification_report(y,y_pred),"\n")
        
    if show_confusion_matrix:
        print ("Confusion matrix")
        print (metrics.confusion_matrix(y,y_pred),"\n")
        
measure_performance(x_test,y_test,clf, show_classification_report=True, show_confusion_matrix=False)

Accuracy:0.512 

Classification report
             precision    recall  f1-score   support

          3       0.00      0.00      0.00         2
          4       0.00      0.00      0.00         4
          5       0.63      0.64      0.63        80
          6       0.44      0.42      0.43        60
          7       0.32      0.55      0.40        11
          8       0.00      0.00      0.00         3

avg / total       0.50      0.51      0.50       160
 



In [63]:
from sklearn.model_selection import cross_val_score, LeaveOneOut, KFold
from scipy.stats import sem

def loo_cv(X_train,y_train,clf):
    # Perform Leave-One-Out cross validation
    # We are preforming 1313 classifications!
    kf = KFold(n_splits=20)
    #loo = LeaveOneOut()
    scores=np.zeros(X_train[:].shape[0])
    for train_index,test_index in kf.split(X_train):
        X_train_cv, X_test_cv= X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv= y_train.iloc[train_index], y_train.iloc[test_index]
        clf = clf.fit(X_train_cv,y_train_cv)
        y_pred=clf.predict(X_test_cv)
        scores[test_index]=metrics.accuracy_score(y_test_cv.astype(int), y_pred.astype(int))
    print (("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores), sem(scores)))

In [64]:
x_data = x_train.append(x_test, ignore_index=True)
y_data = y_train.append(y_test, ignore_index=True)
loo_cv (x_data, y_data, clf)

Mean score: 0.565 (+/-0.001)


In [66]:
columns = data.columns
for feature in zip(columns, clf.feature_importances_):
    print(feature)

('fixed acidity', 0.076667365318003319)
('volatile acidity', 0.13277028273142624)
('citric acid', 0.06829386382289776)
('residual sugar', 0.081226272247025888)
('chlorides', 0.08538803936662466)
('free sulfur dioxide', 0.037970286207025548)
('total sulfur dioxide', 0.12738307810171093)
('density', 0.13661810556910547)
('pH', 0.038817473593324048)
('sulphates', 0.21486523304285607)


In [38]:
clf = clf.fit(x_train,y_train)
measure_performance(x_test,y_test,clf, show_classification_report=True, show_confusion_matrix=False)

Accuracy:0.698 

Classification report
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         1
          1       1.00      1.00      1.00         1
          2       0.50      1.00      0.67         3
          3       1.00      0.33      0.50         3
          4       1.00      1.00      1.00         3
          5       1.00      1.00      1.00         3
          6       0.50      0.67      0.57         3
          7       1.00      1.00      1.00         2
          8       0.75      1.00      0.86         3
          9       0.50      0.33      0.40         3
         10       1.00      1.00      1.00         1
         11       0.75      1.00      0.86         3
         12       1.00      1.00      1.00         1
         13       0.67      1.00      0.80         2
         14       0.00      0.00      0.00         1
         15       0.50      1.00      0.67         2
         16       0.00      0.00      0.00         4
      